Draw a connection map on teams playes against opponents from another league.  
1. Collect the data for matches happen between two teams with different league.
2. Make a plot about connection map that use the link as team play against.

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import os

import plotly.graph_objs as go
import sys
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# helper function to set up connection to database.
def _conn_db(path):
    database = os.path.join(path, 'database.sqlite')
    conn = sqlite3.connect(database)
    return conn
# helper function to get the data.
def _get_table(query,conn):
    df = pd.read_sql(query,conn)
    return df

In [3]:
path=".."
conn = _conn_db(path)

In [4]:
query = "select r3.name as League_name, r.team_long_name as home_team_name1, \
        r.team_short_name as home_team_name2,r2.team_long_name as away_team_name1, r2.team_short_name as \
        away_team_name2,l.season,l.home_team_goal,l.away_team_goal,l.foulcommit,l.possession,l.cross from Match as l left join Team as r \
        on l.home_team_api_id = r.team_api_id \
        left join Team as r2 \
        on l.away_team_api_id=r2.team_api_id\
        left join League as r3\
        on l.league_id = r3.id;"
df = _get_table(query, conn)

In [6]:
df.head()

,League_name,home_team_name1,home_team_name2,away_team_name1,away_team_name2,season,home_team_goal,away_team_goal,foulcommit,possession,cross
0,Belgium Jupiler League,KRC Genk,GEN,Beerschot AC,BAC,2008/2009,1,1,None,None,None
1,Belgium Jupiler League,SV Zulte-Waregem,ZUL,Sporting Lokeren,LOK,2008/2009,0,0,None,None,None
2,Belgium Jupiler League,KSV Cercle Brugge,CEB,RSC Anderlecht,AND,2008/2009,0,3,None,None,None
3,Belgium Jupiler League,KAA Gent,GEN,RAEC Mons,MON,2008/2009,5,0,None,None,None
4,Belgium Jupiler League,FCV Dender EH,DEN,Standard de Liège,STL,2008/2009,1,3,None,None,None


In [31]:
df["Country"] = df["League_name"].apply(lambda cell: cell.split(" ")[0])

In [32]:
df.to_csv("EU_soccer_match_info.csv",header=True)

In [50]:
# heatmap
query = "select * from Team_Attributes;"
team_att_df = _get_table(query,conn)

In [52]:
team_att_df.to_csv("FIFA_team_att.csv",header=True)

goals within a year in terms of leagues, teams. Done in Tableau.  
  
Make a plot on the FIFA's team attributes and the team's actual offensive and defensive performance. The data for offense and defense can be collected from Team Attributes. And we pair them with the actual goal scored in last 3 years.

In [150]:
def _team_FIFA_match(team,conn,season="'2015/2016'"):
    """
    Get the team attributes for either home team or away team combined with match informations
    to assess how well a team's play and its relationship with team build up play.
    param:
        team: the name on team type, either 'home' or 'away'
        conn: to build up connection
    return: 
        df: a dataframe that grouped by league name and team name.
    """
    team_df = _get_table("select * from Team_Attributes;",conn)
    offensive_team_atts = [i  for i in team_df.columns.values if 'Class' not in i][4:10]
    team_att_str = ", r2.".join(offensive_team_atts)
    query = "select r3.name as League_name,l.country_id, l.league_id, l.season, l.home_team_api_id, l.away_team_api_id,\
        l.home_team_goal, l.away_team_goal, r.team_long_name as home_team_name1,{}\
        from Match as l left join Team as r\
        on l.{}_team_api_id = r.team_api_id \
        left join Team_Attributes as r2 on r2.team_api_id = l.{}_team_api_id\
        left join League as r3 on l.league_id = r3.id\
        where (l.season = {});".format(" r2."+team_att_str,team,team,season)
    team_att_match_df = _get_table(query,conn)
    # get the perteam attributes
    df = team_att_match_df.groupby(by=["League_name", "home_team_name1"]).mean()
    home_team_info = df.drop(columns=["country_id","league_id","home_team_api_id","away_team_api_id"])
    return home_team_info

In [153]:
_team_FIFA_match('away',conn).to_csv("away_team.csv",header=True)

In [155]:
_team_FIFA_match('home',conn).to_csv("home_team.csv",header=True)

In [156]:
!open home_team.csv